In [1]:
from VSM import *
from FeatureExtraction import *

In [ ]:
# num_components = min(tfidf_matrix_uc.shape[0], tfidf_matrix_code.shape[1])
# num_components = min(num_components, 100)
# LSA_model = TruncatedSVD(n_components=num_components)

# # if train_or_test == "train":
# LSA_data_useCases = LSA_model.fit_transform(tfidf_matrix_uc)
# LSA_data_codes = LSA_model.fit_transform(tfidf_matrix_code) 
# # else:
# #     LSA_data_useCases = LSA_model.transform(tfidf_matrix_uc)
# #     LSA_data_codes = LSA_model.transform(tfidf_matrix_code) 

# LSA_similraity_matrix = cosine_similarity(LSA_data_useCases, LSA_data_codes)
# print(LSA_similraity_matrix)

In [ ]:
# test = FeatureExtraction()
# test.LSA(tfidf_matrix_uc, tfidf_matrix_code, "train")

In [ ]:
#usecases (100*usecases_nu)
#code (100*codes_nu)
#LSA_similraity_matrix (usecases_nu*codes_nu)
#d1,d2,d3,d4
#u1,u2,u3,u4
#u1*d1,u1*d2,u1*d3 ...
#d1*u1,d1*u2,d1*u3 ...
#each row coresspond between the use cases and each document
# [[d1*c1 d1*c2],[d2*c1 d2*c2]]

# LSA_data_codes=np.array([[1,2,3,4,5] , [6,7,8,9,10]])
# LSA_data_useCases=np.array([[11,12,13,14,15] , [16,17,18,19,21],[2,5,9,11,42]])

# LSA_data_codes_norm=np.repeat(np.linalg.norm(LSA_data_codes,axis=1),LSA_data_useCases.shape[0])
# LSA_data_useCases_norm=np.repeat(np.linalg.norm(LSA_data_useCases,axis=1),LSA_data_codes.shape[0])

# LSA_data_codes_norm=LSA_data_codes_norm.reshape(LSA_data_codes.shape[0],LSA_data_useCases.shape[0]).T 
# LSA_data_useCases_norm=LSA_data_useCases_norm.reshape(LSA_data_useCases.shape[0],LSA_data_codes.shape[0])

# print(LSA_data_codes_norm)
# print(LSA_data_useCases_norm)

# LSA_similraity_matrix=cosine_similarity(LSA_data_useCases, LSA_data_codes)

# print(LSA_similraity_matrix)

## Jensen-Shannon

In [ ]:
test = FeatureExtraction()
test.JensenShannon(UCTokens, UC_documents, code_documents)

In [2]:
#running the query as a whole
feature_extraction=FeatureExtraction(UCTokens)
UC_count_matrix, code_count_matrix = feature_extraction.CountVectorizerModel(UC_documents, code_documents, 'train')

### Run the original query q, and obtain the result list R 
total_score_whole_query_jensen_shannon = feature_extraction.JensenShannon(UC_count_matrix, code_count_matrix) 

### Run each individual query term qt in the original query as a separate query and obtain the result list Rt.
overall_queries_score = list()
for code_idx, code in enumerate(code_documents):
    code_words = code.split(" ")
    query_overlap_list = list()
    for word in code_words:
        if word == '':
            continue
        # word_count_matrix = np.zeros((code_count_matrix.shape[0], code_count_matrix.shape[1]))
        word_index = feature_extraction.code_vocab_index[word]
        # word_count_matrix[:, word_index] = code_count_matrix[:, word_index]
        # print(feature_extraction.count_vectorizer.vocabulary_["if"])
        # print(code_count_matrix[:, 752])
        # print(code_count_matrix[:, word_index])
        term_vector=np.zeros((1, UC_count_matrix.shape[1]))
        if(code_count_matrix[code_idx, word_index] == 0):
            print("here")
            print(word)
            print(code_idx)
        term_vector[:,word_index]=code_count_matrix[code_idx, word_index]
        print("jsdmdjjdmj")
        total_score_query_term_jensen_shannon = feature_extraction.JensenShannon(UC_count_matrix, term_vector)
        print(total_score_query_term_jensen_shannon)

        
    # break




# print(total_score_whole_query_jensen_shannon[:,7])
# print(total_score_whole_query_jensen_shannon[:,6])
# print(code[7])
# for i, temp in enumerate(code[7]):
#     if temp != 0:
#         print(i)
# # 3 features to be called
#1st case => if the query is the code
# total_score_whole_query_jensen_shannon=feature_extraction.JensenShannon(UC_documents,,"train") #58*116

c:\Users\sohad\anaconda3\Lib\site-packages\scipy\sparse\_base.py:659: RuntimeWarning: divide by zero encountered in divide
  recip = np.true_divide(1., other)
c:\Users\sohad\anaconda3\Lib\site-packages\scipy\spatial\distance.py:1250: RuntimeWarning: invalid value encountered in divide
  q = q / np.sum(q, axis=axis, keepdims=True)
